In [ ]:
import kfold_model as kfm
import pandas as pd
import numpy as np
from tensorflow.keras.models import load_model
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt

In [ ]:
# felix function
def outlier_detector_IQR(df,features,n_IQR, all_data = True):  

  dmin = {}
  dmax = {}
  lower_b = {}
  higher_b = {}
  count_min = {}
  count_max = {}

  for i in features:
    Q1 = df[i].quantile(0.25)
    Q3 = df[i].quantile(0.75)
    IQR = Q3 - Q1
    lower_limit, higher_limit = Q1 - n_IQR*IQR, Q3 +n_IQR*IQR

    lower_b[i] = [np.round(lower_limit,2)]   # min value per Log
    higher_b[i] = [np.round(higher_limit,2)] # max value per Log

    if df[i].min() < lower_limit:      
      dmin[i] = [np.round(df[i].min(),2)]
      count_min[i] = [len(df[i].loc[df[i] < lower_limit])]
    else:
      dmin[i] = '-----'
      count_min[i] = '-----'
    if df[i].max() > higher_limit:
      dmax[i] = [np.round(df[i].max(),2)]
      count_max[i] = [len(df[i].loc[df[i] > higher_limit])]
    else:
      dmax[i] = '-----'
      count_max[i] = '-----'

  lower = pd.DataFrame(lower_b).T.rename(columns={0:'Lower_Limit'})
  min = pd.DataFrame(dmin).T.rename(columns={0:'1st_Min_Outlier'})
  min_count = pd.DataFrame(count_min).T.rename(columns={0:'Total_Min_Out'})
  higher = pd.DataFrame(higher_b).T.rename(columns={0:'Higher_Limit'})  
  max = pd.DataFrame(dmax).T.rename(columns={0:'Last_Max_Outlier'})
  max_count = pd.DataFrame(count_max).T.rename(columns={0:'Total_Max_Out'})  

  return pd.concat([lower,min,min_count,higher,max,max_count],axis=1) 

In [ ]:
# felix function
def outlier_replacer(df,column,min=None,max=None):  

  if min != None:
    min_idx = list(df.loc[df[column]<=min].index.values)
    df.at[min_idx,column] = min
  else:
    pass

  if max != None:
    max_idx = list(df.loc[df[column]>=max].index.values)
    df.at[max_idx,column] = max
  else:
    pass  

  return df

## preparing data for the model

In [5]:
# loding eda dataset
well_data_Softypo_ft = pd.read_csv('final_well_data_Softypo_ft.csv', index_col=0)
well_data_Softypo_ft.describe()

,BHT,TrueTemp,DST Test Date,DST Number,BHT_md_ft,BHT_ss_ft,Initial Hydrostatic Pressure (kPa),Final Hydrostatic Pressure (kPa),SurfaceLatitude_NAD83,SurfaceLongitude_NAD83,...,Gas Injection Cum (mcf),GR,ILD,RHO,NPHI,DT,VSHALE,PHIT,SW,U
count,821.000000,615.000000,499.000000,499.000000,821.000000,821.000000,445.000000,442.000000,821.000000,821.000000,...,6.000000,791.000000,616.000000,340.000000,361.000000,533.000000,791.000000,677.000000,5.520000e+02,63.000000
mean,74.143957,90.573978,27006.494990,2.058116,7435.110757,5575.376572,21169.034494,20631.409095,44.712029,-108.719768,...,42166.333333,52.999598,92.405508,2.552946,0.138761,67.918615,0.307140,0.124801,3.004280e-01,10.064914
std,29.367932,29.514626,3995.354937,1.894462,2750.882639,3371.674640,9366.684293,7902.905600,11.952018,9.174035,...,103286.000991,34.866225,240.204006,0.175253,0.122095,16.270341,0.249044,0.106304,3.989270e-01,2.972785
min,10.000000,8.370000,19665.000000,1.000000,739.993462,-2070.045998,2420.060000,2420.060000,27.975343,-118.732890,...,0.000000,1.858000,0.911750,1.769698,-0.006950,9.804400,-0.058157,-0.040799,2.650099e-07,3.821318
25%,51.110000,67.115000,23443.500000,1.000000,5504.080000,2823.162820,16209.580000,16136.407500,30.450949,-116.568760,...,0.000000,25.428525,8.154400,2.469971,0.033500,54.335800,0.110204,0.038042,9.651705e-03,7.967110
50%,72.222222,90.555556,27019.000000,1.000000,7291.502858,5190.124838,19339.800000,19179.500000,53.487604,-115.083627,...,0.000000,45.332675,18.406900,2.597964,0.106000,65.115250,0.252376,0.103116,5.935690e-02,10.404690
75%,91.111111,110.380000,29654.500000,2.000000,9056.922862,7894.000000,25026.000000,24800.500000,54.539870,-97.579842,...,0.000000,74.298788,52.854450,2.678262,0.226700,77.596350,0.459277,0.191287,6.061640e-01,11.734704
max,196.111111,196.027778,37913.000000,16.000000,19502.980000,19132.000000,120704.000000,62162.000000,56.050945,-95.623436,...,252998.000000,282.699650,2003.953100,2.913450,0.547900,143.443250,1.947855,0.628893,1.000000e+00,21.161124


In [ ]:
# looking for outliers
outlier_detector_IQR(well_data_Softypo_ft[well_data_Softypo_ft.dtypes[well_data_Softypo_ft.dtypes != 'object'].index],well_data_Softypo_ft[well_data_Softypo_ft.dtypes[well_data_Softypo_ft.dtypes != 'object'].index].keys(),1.5)

In [6]:
# droping outlier values
outlier_replacer(well_data_Softypo_ft,'Gas Maximum (mcf)',min=0,max=19436.24)
outlier_replacer(well_data_Softypo_ft,'Oil Total Cum (bbl)',min=0,max=14803.74)
outlier_replacer(well_data_Softypo_ft,'Initial Hydrostatic Pressure (kPa)',min=0,max=39112.16)
outlier_replacer(well_data_Softypo_ft,'Water Maximum (bbl)',min=0,max=1565.31) 
outlier_replacer(well_data_Softypo_ft,'Gas Total Cum (mcf)',min=0,max=227782.69) 
outlier_replacer(well_data_Softypo_ft,'Final Hydrostatic Pressure (kPa)',min=0,max=38256.44)  
outlier_replacer(well_data_Softypo_ft,'Oil Maximum (bbl)',min=0,max=3295.82) 
outlier_replacer(well_data_Softypo_ft,'GOR Total Average',min=0,max=5663.28)
outlier_replacer(well_data_Softypo_ft,'Water Total Cum (bbl)',min=0,max=66035.43) 
outlier_replacer(well_data_Softypo_ft,'TD_ft',min=None,max=15830.82)
outlier_replacer(well_data_Softypo_ft,'Total Vertical Depth (ft)',min=None,max=15231.25) 
outlier_replacer(well_data_Softypo_ft,'VSHALE',min=0,max=0.95) 
outlier_replacer(well_data_Softypo_ft,'PHIT',min=0,max=None) 
outlier_replacer(well_data_Softypo_ft,'ILD',min=0,max=2.97) 
outlier_replacer(well_data_Softypo_ft,'DT',min=20.28,max=112.42)
outlier_replacer(well_data_Softypo_ft,'GR',min=0,max=143.62) 
outlier_replacer(well_data_Softypo_ft,'U',min=None,max=16.16) 
print ('done')

done


In [7]:
well_data_Softypo_ft.describe()

,BHT,TrueTemp,DST Test Date,DST Number,BHT_md_ft,BHT_ss_ft,Initial Hydrostatic Pressure (kPa),Final Hydrostatic Pressure (kPa),SurfaceLatitude_NAD83,SurfaceLongitude_NAD83,...,Gas Injection Cum (mcf),GR,ILD,RHO,NPHI,DT,VSHALE,PHIT,SW,U
count,821.000000,615.000000,499.000000,499.000000,821.000000,821.000000,445.000000,442.000000,821.000000,821.000000,...,6.000000,791.000000,616.000000,340.000000,361.000000,533.000000,791.000000,677.000000,5.520000e+02,63.000000
mean,74.143957,90.573978,27006.494990,2.058116,7435.110757,5575.376572,20844.464764,20497.702783,44.712029,-108.719768,...,42166.333333,52.383657,2.930069,2.552946,0.138761,67.693304,0.303183,0.125120,3.004280e-01,9.942615
std,29.367932,29.514626,3995.354937,1.894462,2750.882639,3371.674640,7692.482715,7480.535569,11.952018,9.174035,...,103286.000991,32.552215,0.214640,0.175253,0.122095,15.355093,0.231544,0.105889,3.989270e-01,2.605004
min,10.000000,8.370000,19665.000000,1.000000,739.993462,-2070.045998,2420.060000,2420.060000,27.975343,-118.732890,...,0.000000,1.858000,0.911750,1.769698,-0.006950,20.280000,0.000000,0.000000,2.650099e-07,3.821318
25%,51.110000,67.115000,23443.500000,1.000000,5504.080000,2823.162820,16209.580000,16136.407500,30.450949,-116.568760,...,0.000000,25.428525,2.970000,2.469971,0.033500,54.335800,0.110204,0.038042,9.651705e-03,7.967110
50%,72.222222,90.555556,27019.000000,1.000000,7291.502858,5190.124838,19339.800000,19179.500000,53.487604,-115.083627,...,0.000000,45.332675,2.970000,2.597964,0.106000,65.115250,0.252376,0.103116,5.935690e-02,10.404690
75%,91.111111,110.380000,29654.500000,2.000000,9056.922862,7894.000000,25026.000000,24800.500000,54.539870,-97.579842,...,0.000000,74.298788,2.970000,2.678262,0.226700,77.596350,0.459277,0.191287,6.061640e-01,11.734704
max,196.111111,196.027778,37913.000000,16.000000,19502.980000,19132.000000,39112.160000,38256.440000,56.050945,-95.623436,...,252998.000000,143.620000,2.970000,2.913450,0.547900,112.420000,0.950000,0.628893,1.000000e+00,16.160000


In [ ]:
# % of nan
well_data_Softypo_ft.isnull().mean()

In [ ]:
#droping columns with nan higer than 50%
well_data_Softypo_ft = well_data_Softypo_ft.loc[:, well_data_Softypo_ft.isnull().mean() < .5]

In [8]:
well_data_Softypo_ft.columns

Index(['Set', 'source', 'BHT', 'TrueTemp', 'Field', 'DST Test Date',
       'Test Type', 'DST Misrun', 'DST Number', 'formation', 'BHT_md_ft',
       'BHT_ss_ft', 'Initial Hydrostatic Pressure (kPa)',
       'Final Hydrostatic Pressure (kPa)', 'SurfaceLatitude_NAD83',
       'SurfaceLongitude_NAD83', 'BottomLatitude_NAD83',
       'BottomLongitude_NAD83', 'Elevation_KB_ft', 'TD_ft',
       'Total Vertical Depth (ft)', 'Spud Date', 'Completion Date',
       'First Production Month', 'Oil Total Cum (bbl)', 'Gas Total Cum (mcf)',
       'Water Total Cum (bbl)', 'GOR Total Average', 'Plug Date',
       'First Production Date', 'Last Production Month', 'Gas Maximum (mcf)',
       'Gas Maximum Date', 'Oil Maximum (bbl)', 'Oil Maximum Date',
       'Water Maximum (bbl)', 'Water Maximum Date', 'Yield Total Average',
       'TSC or ORT (time since circulation or original recorded time in hours)',
       'MinCasingSize', 'spuddate', 'completiondate', 'cumoil', 'cumgas',
       'cumwater', 'Mud W

In [17]:
df = pd.DataFrame(well_data_Softypo_ft.loc[:,['source', 'BHT', 'TrueTemp', 'Field', 'BHT_md_ft',
       'BHT_ss_ft', 'Initial Hydrostatic Pressure (kPa)',
       'Final Hydrostatic Pressure (kPa)', 'SurfaceLatitude_NAD83',
       'SurfaceLongitude_NAD83', 'Elevation_KB_ft', 'TD_ft', 'Oil Total Cum (bbl)', 'Gas Total Cum (mcf)',
       'Water Total Cum (bbl)', 'GOR Total Average', 'Gas Maximum (mcf)', 'Yield Total Average',
       'TSC or ORT (time since circulation or original recorded time in hours)',
       'MinCasingSize', 'completiondate', 'cumoil', 'cumgas',
       'cumwater', 'Mud Wt', 'MW@Depth(KB)', 'GR', 'ILD',
       'RHO', 'NPHI', 'DT', 'VSHALE', 'PHIT', 'SW', 'U']], index=well_data_Softypo_ft.index)

# casting categorilac data
df[df.dtypes[df.dtypes == 'object'].index] = df[df.dtypes[df.dtypes == 'object'].index].astype('category')

In [ ]:
# selecting working variables for TVD estimation from WellHeader_Datathon.csv
df = pd.DataFrame(well_data_Softypo_ft.loc[:,['source', 'BHT', 'Field', 'TrueTemp', 'BHT_md_ft', 'SurfaceLatitude_NAD83', 'SurfaceLongitude_NAD83', 'VSHALE', 'PHIT', 'SW']], index=well_data_Softypo_ft.index)

# casting categorilac data
df[df.dtypes[df.dtypes == 'object'].index] = df[df.dtypes[df.dtypes == 'object'].index].astype('category')

In [18]:
df

,source,BHT,TrueTemp,Field,BHT_md_ft,BHT_ss_ft,Initial Hydrostatic Pressure (kPa),Final Hydrostatic Pressure (kPa),SurfaceLatitude_NAD83,SurfaceLongitude_NAD83,...,MW@Depth(KB),GR,ILD,RHO,NPHI,DT,VSHALE,PHIT,SW,U
UWI,,,,,,,,,,,,,,,,,,,,,
42021301990000,synthetic,73.333333,NaN,Eaglebine,6910.000000,6588.000000,NaN,NaN,30.011031,-97.127885,...,6910.0,73.422600,2.9700,2.576150,0.185000,NaN,0.453019,0.131637,0.087182,NaN
100102606420W500,synthetic,27.780000,NaN,Duvernay,4887.467348,2287.401648,17871.21,17705.74,54.568936,-116.909223,...,NaN,48.028750,2.3187,NaN,NaN,112.420000,0.271634,0.493585,0.007834,NaN
100141705519W500,synthetic,83.330000,NaN,Duvernay,8177.493700,4425.525076,27634.20,26855.09,53.756720,-116.790110,...,NaN,71.541450,2.9700,NaN,NaN,92.898700,0.439582,0.297236,0.007249,NaN
100141503621W400,synthetic,48.890000,NaN,Duvernay,4317.093314,1339.074846,14927.15,14927.15,52.097198,-112.924817,...,NaN,46.398625,2.9700,NaN,NaN,85.880150,0.259990,0.251182,0.013685,NaN
100043406718W500,synthetic,65.560000,NaN,Duvernay,7266.568474,4725.557894,24765.97,24441.92,54.837612,-116.667754,...,NaN,19.313100,2.9700,NaN,NaN,53.188850,0.066522,0.036672,1.000000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100101606423W500,synthetic,47.220000,56.340000,Duvernay,4890.092020,2243.110308,16581.90,16581.90,54.539870,-117.413684,...,NaN,126.556100,2.9700,2.045650,0.387500,80.213800,0.832544,0.330003,0.004688,NaN
42177309850000,synthetic,74.444444,96.655556,Eaglebine,7032.760000,6677.000000,NaN,NaN,29.287122,-97.693154,...,7035.0,10.848950,2.9700,NaN,0.014200,NaN,0.006064,0.014200,1.000000,NaN
100100805919W500,synthetic,69.440000,73.050000,Duvernay,6631.889976,3643.044736,21518.54,21146.23,54.089493,-116.809579,...,NaN,78.612650,2.9700,2.526800,0.243900,72.978800,0.490090,0.172521,0.015688,NaN


In [19]:
df_num = df[df.dtypes[df.dtypes != 'category'].index]

In [20]:
# one hot encoder for categories
ohe = OneHotEncoder(sparse=False)
df_ohe = pd.DataFrame(ohe.fit_transform(df[df.dtypes[df.dtypes == 'category'].index]), columns=ohe.get_feature_names(df.dtypes[df.dtypes == 'category'].index), index=df.index)
# adding numerical columns
df_ohe = pd.concat([df[df.dtypes[df.dtypes != 'category'].index], df_ohe], axis=1, verify_integrity=True)

In [21]:
# generating train and target dataframes
df_train = df_ohe[~df_ohe.TrueTemp.isna()]
# target datasets
df_target = df_ohe[df_ohe.TrueTemp.isna()]

In [22]:
# spliting the training and validation data
train_x, holdout_x, train_y, holdout_y = train_test_split(df_train.loc[:, df_train.columns != 'TrueTemp'], df_train.TrueTemp, test_size=0.1, random_state=42)

# filling nan values
train_x = train_x.fillna(0)
holdout_x = holdout_x.fillna(0)

# features scaling
sc = StandardScaler()
train_x = sc.fit_transform(train_x)
holdout_x = sc.transform(holdout_x)

In [23]:
# saving train and validation sets in binary format
np.save('train_x.npy', train_x)
np.save('holdout_x.npy', holdout_x)
np.save('train_y.npy', train_y)
np.save('holdout_y.npy', holdout_y)

In [ ]:
# loading train and validation sets in binary format
train_x = np.load('train_x.npy')
holdout_x = np.load('holdout_x.npy')
train_y = np.load('train_y.npy')
holdout_y = np.load('holdout_y.npy')

In [24]:
print ('train_x: ',train_x.shape)
print ('holdout_x: ',holdout_x.shape)

train_x:  (553, 293)
holdout_x:  (62, 293)


## implementing k-folds model

In [ ]:
models, histories, hscores = kfm.model_kfold(train_x, train_y, holdout=[holdout_x,holdout_y], model='v1', num_folds=5, batch_size=None, steps_per_epoch=1, loss_function='mean_squared_error', optimizer='adam', max_epochs=500, verbosity=1, workers=6, use_multiprocessing=True, continue_training=False, save_models_afte_training=True, plot_results=True, reshuffle=False, random_state=42, path=None)

In [ ]:
kfm.graph(histories, tight_layout=True, holdoutscores=hscores)

In [ ]:
# save model
#model.save('./_model_deep/model.h5')

In [ ]:
yhat = models['k4_model'].predict(holdout_x)

RMSE = mean_squared_error(yhat, holdout_y, squared=False)

# poly1d_fn is a function which takes in yhat values and returns an estimate for y
coef = np.polyfit(yhat.flat, holdout_y,1)
poly1d_fn = np.poly1d(coef)

# correlation y/yhat
plt.scatter(yhat, holdout_y, color='Red')
plt.plot(yhat , poly1d_fn(yhat), '--k', linewidth=3,)
plt.xlabel("yhat (temp)")
plt.ylabel("y (temp)")
plt.text(0, 0, r'RMSE: {:.4f}'.format(RMSE), fontsize=15)
plt.show()

## preparing submission dataframe

In [ ]:
# creating submission dataframe
df_target = df_target.fillna(0)
df_target['TrueTemp'] = models['k4_model'].predict(sc.transform(df_target.loc[:, df_target.columns != 'TrueTemp']))
df_to_send = df_target.reset_index()
df_to_send[['UWI', 'TrueTemp']].to_csv('predictions.csv', index=True)

In [ ]:
# creating submission dataframe
df_target = df_target.fillna(0)
df_target['k1'] = models['k1_model'].predict(sc.transform(df_target.loc[:, df_target.columns != 'TrueTemp']))
df_target['k2'] = models['k2_model'].predict(sc.transform(df_target.loc[:, df_target.columns != 'TrueTemp']))
df_target['k3'] = models['k3_model'].predict(sc.transform(df_target.loc[:, df_target.columns != 'TrueTemp']))
df_target['k4'] = models['k4_model'].predict(sc.transform(df_target.loc[:, df_target.columns != 'TrueTemp']))
df_target['k5'] = models['k5_model'].predict(sc.transform(df_target.loc[:, df_target.columns != 'TrueTemp']))
df_target['TrueTemp'] = (df_target['k1']+df_target['k2']+df_target['k3']+df_target['k3']+df_target['k5'])/5
df_to_send = df_target.reset_index()
df_to_send[['UWI', 'TrueTemp']].to_csv('predictions.csv', index=True)

In [ ]:
# ziping submission
import zipfile
zipfile.ZipFile('predictions.zip', mode='w').write("predictions.csv")